### Load preprocessed data

Run the script that downloads and processes the MovieLens data.
Uncomment it to run the download & processing script.

In [1]:
#!python ../src/download.py

In [1]:
import numpy as np
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])

In [2]:
from abstract_model import AbstractModel

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, n_occu, k=18, c_ovector=1.0,
                 c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.c_ovector = c_ovector
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        
        # **NEW: occupation vectors
        self.occu = nn.Embedding(n_occu, k)

    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # NEW: occupation-item interaction
        occu_id = inputs[:, 3]
        vector_occu = self.occu(occu_id)
        oi_interaction = torch.sum(vector_user * vector_occu, dim=1)
        
        prediction = ui_interaction + oi_interaction + biases
        return prediction

    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        log = {"mse": loss_mse}
        return loss_mse, log
    
    def reg(self):
        # Add new regularization to the biases
        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        
        # Compute new occupation regularization
        reg_occu = l2_regularize(self.occu.weight) * self.c_ovector
        
        log = {"reg_user": reg_user, "reg_item": reg_item,
               "reg_bias_user": reg_bias_user, "reg_bias_item": reg_bias_item,
               "reg_occu": reg_occu
              }
        total = reg_user + reg_item + reg_bias_user + reg_bias_item + reg_occu
        return total, log

In [4]:
import optuna
from pytorch_lightning.loggers.wandb import WandbLogger


k = 6
c_bias = 1e-3
c_vector = 1e-5
c_ovector = 1e-8
model = MF(n_user, n_item, n_occu, 
           k=k, c_bias=c_bias, c_vector=c_vector,
           c_ovector=c_ovector,
           batch_size=1024)
model.save_data(train_x, train_y, test_x, test_y)

# add a logger
logger = WandbLogger(name="03_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     gpus=1,
                     progress_bar_refresh_rate=1) 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


### Train model

In [5]:
trainer.fit(model)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: sf-moody (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200925_152942-sdxgg8cr
wandb: Syncing run 03_mf



  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 36 K  
1 | item      | Embedding | 23 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   
4 | occu      | Embedding | 126   


Saving latest checkpoint..


1

### Test Model

In [ ]:
results = trainer.test(model)
mse = results['avg_test_loss']
rmse = np.sqrt(mse)
rmse

--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(18.0658, device='cuda:0'),
 'val_loss': tensor(18.0658, device='cuda:0')}
--------------------------------------------------------------------------------



4.2503886325643245